Integrantes:

Castillo Ilabaca, Sebastian
Corti, Gastón

Enunciado:

1 ¿Cuáles son los PEAS de este problema? (Performance, Environment, Actuators, Sensors)

Performance // Numero de movimientos realizados para llegar al estado final, tiempo de ejecucion de codigo para llegar al estado final, uso de memoria.
Enviroment // Entorno de desarrollo
Actuators // El codigo que identifica las posiciones inicial y final y resuelve la cantidad de movimientos necesarios para resolver el problema.
Sensors // 

2 ¿Cuáles son las propiedades del entorno de trabajo?

Totalmente observable vs parcialmente observable //
Determinista vs estocastico //
Episodico vs secuancial //
Discreto vs continuo //
Agente individual vs multiagente (competitivo o cooperativo) //

3 En el contexto de este problema, establezca cuáles son los: estado, espacio de estados, árbol de búsqueda, nodo de búsqueda, objetivo, acción y frontera.

4 Implemente algún método de búsqueda. Puedes elegir cualquiera menos búsqueda en anchura primero (el desarrollado en clase). Sos libre de elegir cualquiera de los vistos en clases, o inclusive buscar nuevos.

5 ¿Qué complejidad en tiempo y memoria teórica tiene el algoritmo elegido? 

6 A nivel implementación, ¿qué tiempo y memoria ocupa el algoritmo? (Se recomienda correr 10 veces y calcular promedio y desvío estándar de las métricas).

7 Si la solución óptima es 2k -1 movimientos con k igual al número de discos. Qué tan lejos está la solución del algoritmo implementado de esta solución óptima (se recomienda correr al menos 10 veces y usar el promedio de trayecto usado).

IMPLEMENTACIÓN DE METODO DE BUSQUEDA EN PROFUNDIDAD (COLA LIFO)

In [1]:
import sys
# Add the path to the module
sys.path.append(r'C:\Users\sebas\Hanoi_Tower_TP1\intro_ia\clase2\hanoi_tower')

# Import the desired components from the module
from aima_libs.hanoi_states import StatesHanoi
from aima_libs.hanoi_states import ActionHanoi
from aima_libs.hanoi_states import ProblemHanoi
from aima_libs.tree_hanoi import NodeHanoi

In [2]:
def depth_first_search(number_disks=5):
    # Defining the problem
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Defining the root
    root = NodeHanoi(state=initial_state)
    
    # Defining the LIFO queue (frontier)
    frontier = [NodeHanoi(problem.initial)]

    # Defining the set of visited states
    explored = set()

    node_explored = 0

    while len(frontier) != 0:
        node = frontier.pop()
        node_explored += 1

        # Adding the explored node to the set
        explored.add(node.state)

        # Checking if the goal has been reached
        if problem.goal_test(node.state):
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,  # Assuming this exists
            }
            return node, metrics

        for next_node in node.expand(problem):
            if next_node.state not in explored and next_node not in frontier:
                frontier.append(next_node)

    # Metrics for unsuccessful search
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth if len(frontier) > 0 else 0,
        "cost_total": None,
    }
    return None, metrics


In [3]:
solution, metrics = depth_first_search(number_disks=5)

In [4]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 82
states_visited: 82
nodes_in_frontier: 41
max_depth: 81
cost_total: 81.0


In [5]:
for nodos in solution.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 | 2 | 3 1>
<Node HanoiState: 5 4 2 |  | 3 1>
<Node HanoiState: 5 4 2 | 1 | 3>
<Node HanoiState: 5 4 | 1 | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 | 4 | 3 1>
<Node HanoiState: 5 | 4 2 | 3 1>
<Node HanoiState: 5 | 4 2 1 | 3>
<Node HanoiState: 5 3 | 4 2 1 | >
<Node HanoiState: 5 3 | 4 2 | 1>
<Node HanoiState: 5 3 2 | 4 | 1>
<Node HanoiState: 5 3 2 | 4 1 | >
<Node HanoiState: 5 3 | 4 1 | 2>
<Node HanoiState: 5 3 | 4 | 2 1>
<Node HanoiState: 5 | 4 3 | 2 1>
<Node HanoiState: 5 | 4 3 1 | 2>
<Node HanoiState: 5 2 | 4 3 1 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState:  | 4 3 2 | 5 1

In [6]:
for act in solution.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from

Medicion de performance

In [7]:
%%timeit 
solution, metrics = depth_first_search(number_disks=5)

15.5 ms ± 2.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
import tracemalloc

tracemalloc.start()

solution, metrics = depth_first_search(number_disks=5)

# Para medir memoria consumida usamos el pico de memoria
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Pico de memoria ocupada: {round(memory_peak, 2)} [MB]", )

Pico de memoria ocupada: 0.17 [MB]


In [10]:
import numpy as np  # Para calcular promedio y desviación estándar

def compare_with_optimal(number_disks=5, runs=10):
    optimal_solution = 2**number_disks - 1  # Fórmula para solución óptima
    moves_used = []  # Lista para almacenar los movimientos usados

    for _ in range(runs):
        solution, metrics = depth_first_search(number_disks=number_disks)
        if metrics["solution_found"]:
            moves_used.append(metrics["cost_total"])  # Suponiendo que cost_total almacena movimientos totales
    
    # Calcular promedio y desviación estándar
    avg_moves = np.mean(moves_used)
    std_dev_moves = np.std(moves_used)
    difference_from_optimal = avg_moves - optimal_solution  # Diferencia respecto a la solución óptima

    results = {
        "optimal_solution": optimal_solution,
        "average_moves": avg_moves,
        "std_dev_moves": std_dev_moves,
        "difference_from_optimal": difference_from_optimal,
    }

    return results

# Ejecutar la comparación
results = compare_with_optimal(number_disks=5, runs=10)

# Imprimir los resultados
for metric, value in results.items():
    print(f"{metric}: {value}")


optimal_solution: 31
average_moves: 81.0
std_dev_moves: 0.0
difference_from_optimal: 50.0
